# Output Parsers for LLM Input / Output with LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [1]:
!pip install langchain==0.1.19
!pip install langchain-openai==0.1.6
!pip install langchain-community==0.0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [2]:
from getpass import getpass

OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

Please enter your Open AI API Key here: ··········


## Setup necessary system environment variables

In [3]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

## Output Parsers
Output parsers are essential in Langchain for structuring the responses from language models. Below, we will discuss the role of output parsers and include examples using Langchain's specific parser types: PydanticOutputParser, JsonOutputParser, and CommaSeparatedListOutputParser.

- **Pydantic parser:**
  - This parser allows the specification of an arbitrary Pydantic Model to query LLMs for outputs matching that schema. Pydantic's BaseModel functions similarly to a Python dataclass but includes type checking and coercion.

- **JSON parser:**
  - Users can specify an arbitrary JSON schema with this parser to ensure outputs from LLMs adhere to that schema. Pydantic can also be used to declare your data model here.

- **CSV parser:**
  - Useful for outputs requiring a list of items separated by commas. This parser facilitates the extraction of comma-separated values from model outputs.


### PydanticOutputParser

This output parser allows users to specify an arbitrary Pydantic Model and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

Use Pydantic to declare your data model. Pydantic's BaseModel is like a Python dataclass, but with actual type checking + coercion.

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [6]:
# langchain pre-generated output response formatting instructions
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"title": "Description", "description": "A brief description of the topic asked by the user", "type": "string"}, "pros": {"title": "Pros", "description": "3 bullet points showing the pros of the topic asked by the user", "type": "string"}, "cons": {"title": "Cons", "description": "3 bullet points showing the cons of the topic asked by the user", "type": "string"}, "conclusion": {"title": "Conclusion", "description": "One line conclusion of the topic asked by the user", "type": "string"}}, "required": ["descriptio

In [7]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "A brief description of the topic asked by the user", "type": "string"}, "pros": {"title": "Pros", "description": "3 bullet points showing the pros of the topic asked by the user", "type": "string"}, "cons": {"title": "Cons", "description": "3 bullet points showing the cons of the topic asked by the user", "type": "string"}, "conclusion": {"title": "Conclusion", "description": "One l

In [8]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
           |
         chatgpt
           |
         parser)

In [9]:
question = "Tell me about Commercial Real Estate"
response = chain.invoke({"query": question})

In [10]:
response

QueryResponse(description='Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.', pros='1. Potential for high returns on investment\n2. Diversification of investment portfolio\n3. Long-term lease agreements provide stable income', cons='1. Market fluctuations can impact property values\n2. High upfront costs for purchasing commercial properties\n3. Vacancy rates can affect cash flow', conclusion='Commercial real estate can be a lucrative investment option for those willing to take on the risks associated with it.')

In [11]:
response.description

'Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.'

In [12]:
response.dict()

{'description': 'Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.',
 'pros': '1. Potential for high returns on investment\n2. Diversification of investment portfolio\n3. Long-term lease agreements provide stable income',
 'cons': '1. Market fluctuations can impact property values\n2. High upfront costs for purchasing commercial properties\n3. Vacancy rates can affect cash flow',
 'conclusion': 'Commercial real estate can be a lucrative investment option for those willing to take on the risks associated with it.'}

In [13]:
for k,v in response.dict().items():
    print(f"{k}:\n{v}\n")

description:
Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.

pros:
1. Potential for high returns on investment
2. Diversification of investment portfolio
3. Long-term lease agreements provide stable income

cons:
1. Market fluctuations can impact property values
2. High upfront costs for purchasing commercial properties
3. Vacancy rates can affect cash flow

conclusion:
Commercial real estate can be a lucrative investment option for those willing to take on the risks associated with it.



### JsonOutputParser

This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

It is recommended use Pydantic to declare your data model.

In [14]:
from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=QueryResponse)
parser

JsonOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [15]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "A brief description of the topic asked by the user", "type": "string"}, "pros": {"title": "Pros", "description": "3 bullet points showing the pros of the topic asked by the user", "type": "string"}, "cons": {"title": "Cons", "description": "3 bullet points showing the cons of the topic asked by the user", "type": "string"}, "conclusion": {"title": "Conclusion", "description": "One l

In [16]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
              |
            chatgpt
              |
            parser)
chain

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"title": "Description", "description": "A brief description of the topic asked by the user", "type": "string"}, "pros": {"title": "Pros", "description": "3 bullet points showing the pros of the topic asked by the user", "type": "string"}, "cons": {"title": "Cons", "description": "3 bullet points showing the cons of the topic asked by the user", "type": "string"}, "conclusion": {"title": "Conclusion", "description": "One l

In [17]:
topic_queries = [
    "Tell me about commercial real estate",
    "Tell me about Generative AI"
]

topic_queries_formatted = [{"query": topic}
                    for topic in topic_queries]
topic_queries_formatted

[{'query': 'Tell me about commercial real estate'},
 {'query': 'Tell me about Generative AI'}]

In [18]:
responses = chain.map().invoke(topic_queries_formatted)

In [19]:
responses[0], type(responses[0])

({'description': 'Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.',
  'pros': '1. Potential for high returns on investment\n2. Diversification of investment portfolio\n3. Long-term lease agreements provide stable income',
  'cons': '1. Market fluctuations can impact property values\n2. High upfront costs for purchasing commercial properties\n3. Vacancy rates can affect rental income',
  'conclusion': 'Commercial real estate can be a lucrative investment option but requires careful research and management to mitigate risks.'},
 dict)

In [20]:
import pandas as pd

df = pd.DataFrame(responses)
df

,description,pros,cons,conclusion
0,Commercial real estate refers to properties us...,1. Potential for high returns on investment\n2...,1. Market fluctuations can impact property val...,Commercial real estate can be a lucrative inve...
1,Generative AI refers to a type of artificial i...,1. Can be used to generate realistic images an...,1. May raise ethical concerns regarding the au...,Generative AI shows great promise in revolutio...


In [21]:
for response in responses:
  for k,v in response.items():
    print(f"{k}:\n{v}\n")
  print('-----')

description:
Commercial real estate refers to properties used for business purposes, such as office buildings, retail spaces, and industrial facilities. It involves leasing, buying, or selling properties for commercial use.

pros:
1. Potential for high returns on investment
2. Diversification of investment portfolio
3. Long-term lease agreements provide stable income

cons:
1. Market fluctuations can impact property values
2. High upfront costs for purchasing commercial properties
3. Vacancy rates can affect rental income

conclusion:
Commercial real estate can be a lucrative investment option but requires careful research and management to mitigate risks.

-----
description:
Generative AI refers to a type of artificial intelligence that is capable of creating new content, such as images, text, or music, based on patterns and data it has been trained on.

pros:
1. Can be used to generate realistic images and videos for various applications. 2. Can assist in creating personalized conten

### CommaSeparatedListOutputParser

This output parser can be used when you want to return a list of comma-separated items.

In [22]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [23]:
format_instructions = output_parser.get_format_instructions()

# And a query intented to prompt a language model to populate the data structure.
prompt_txt = """
             Create a list of 5 different ways in which Generative AI can be used

             Output format instructions:
             {format_instructions}
             """

prompt = PromptTemplate.from_template(template=prompt_txt)
prompt

PromptTemplate(input_variables=['format_instructions'], template='\n             Create a list of 5 different ways in which Generative AI can be used\n\n             Output format instructions:\n             {format_instructions}\n             ')

In [24]:
# create a simple LLM Chain - more on this later
llm_chain = (prompt
              |
            chatgpt
              |
            output_parser)

# run the chain
response = llm_chain.invoke({'format_instructions': format_instructions})
response

['Art generation',
 'Music composition',
 'Text generation',
 'Video game design',
 'Fashion design']

In [25]:
type(response)

list